In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [81]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=50, UNITS=220):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = model 
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 64, 50)       2200        input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_10 (Bidirectional (None, 64, 220)      476960      embedding_5[0][0]                
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 64, 220)      776160      bidirectional_10[0][0]           
____________________________________________________________________________________________

In [38]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [39]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [88]:
data_other = load_data(['biblical', 'garbage', 'poetry'])

In [89]:
data_rabanit = load_data(['rabanit'])

In [90]:
data_pre_modern = load_data(['pre_modern'])

In [91]:
data_modern = load_data(validation=0.2, source=['modern'])

In [92]:
model.load_weights('./checkpoints/uninit')
history = fit(data_other, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/other')

Train on 49756 samples, validate on 7 samples
49756/49756 [==============================] - 62s 1ms/sample - loss: 0.5692 - N_loss: 0.4514 - D_loss: 0.0996 - S_loss: 0.0182 - N_accuracy: 0.8430 - D_accuracy: 0.9607 - S_accuracy: 0.9950 - val_loss: 0.5293 - val_N_loss: 0.4170 - val_D_loss: 0.1026 - val_S_loss: 0.0097 - val_N_accuracy: 0.8794 - val_D_accuracy: 0.9724 - val_S_accuracy: 0.9950


In [93]:
model.load_weights('./checkpoints/other')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 250e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

Train on 69430 samples, validate on 12571 samples
69430/69430 [==============================] - 89s 1ms/sample - loss: 0.2198 - N_loss: 0.1729 - D_loss: 0.0438 - S_loss: 0.0032 - N_accuracy: 0.9409 - D_accuracy: 0.9836 - S_accuracy: 0.9991 - val_loss: 0.3994 - val_N_loss: 0.2820 - val_D_loss: 0.0875 - val_S_loss: 0.0298 - val_N_accuracy: 0.9204 - val_D_accuracy: 0.9757 - val_S_accuracy: 0.9982


In [94]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

Train on 65636 samples, validate on 6892 samples
65636/65636 [==============================] - 82s 1ms/sample - loss: 0.3199 - N_loss: 0.2463 - D_loss: 0.0653 - S_loss: 0.0083 - N_accuracy: 0.9170 - D_accuracy: 0.9745 - S_accuracy: 0.9979 - val_loss: 0.3099 - val_N_loss: 0.2062 - val_D_loss: 0.0959 - val_S_loss: 0.0077 - val_N_accuracy: 0.9330 - val_D_accuracy: 0.9636 - val_S_accuracy: 0.9982


In [95]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.1e-3))
# history = fit(data_modern, scheduler=CircularLearningRate(1e-3, 7e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

Train on 14133 samples, validate on 3543 samples
14133/14133 [==============================] - 27s 2ms/sample - loss: 0.2350 - N_loss: 0.1745 - D_loss: 0.0563 - S_loss: 0.0042 - N_accuracy: 0.9414 - D_accuracy: 0.9786 - S_accuracy: 0.9990 - val_loss: 0.2263 - val_N_loss: 0.1661 - val_D_loss: 0.0568 - val_S_loss: 0.0034 - val_N_accuracy: 0.9443 - val_D_accuracy: 0.9785 - val_S_accuracy: 0.9991


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [87]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [97]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)

print_predictions(data_modern[1], 5)

מצוי:  וְעָתִיד מְשׁוּתָּף בֵּין הַחֲרֵדִים לְצִיּוֹנִים הוּא חֵלֶק מִמְצִיאוּת הַחַיִּים הַיִּשְׂרְאֵלִית 
רצוי:  וְעָתִיד מְשׁוּתָּף בֵּין הַחֲרֵדִים לַצִּיּוֹנִים הוּא חֵלֶק מִמְּצִיאוּת הַחַיִּים הַיִּשְׂרְאֵלִית 
מצוי:  אַרְצֶה אוֹתָךָ" אוֹ "אֲנִי אֶגְרוֹם לְךָ לְהַרְגִּישׁ נִפְלָא". הִתִּרְגּוּם הָעִבְרִי לְמִילָּה 
רצוי:  אֶרְצֶה אוֹתְךָ" אוֹ "אֲנִי אֶגְרוֹם לְךָ לְהַרְגִּישׁ נִפְלָא". הַתַּרְגּוּם הָעִבְרִי לְמִילָּה 
מצוי:  זְמַן לְהִתְאַרְגֵן עַל כֶּסֶף כְּדֵי לְשַׁלֵּם לְמִי שֶׁמּוֹצִיאִים אֶת סִירוּת הַגּוּמִי הָאֵלֶּה. 
רצוי:  זְמַן לְהִתְאַרְגֵּן עַל כֶּסֶף כְּדֵי לְשַׁלֵּם לְמִי שֶׁמּוֹצִיאִים אֶת סִירוֹת הַגּוּמִי הָאֵלֶּה. 
מצוי:  חֲזְרָה בָּרַגְלַיִם קַלּוֹת. צִפּוֹר צִבְעוֹנִית לוֹתָהּ אוֹתָהּ בְּשִׁיר.
רצוי:  חֲזָרָה בְּרַגְלַיִם קַלוֹת. צִפּוֹר צִבְעוֹנִית לִוְתָה אוֹתָה בְּשִׁיר.
מצוי:  רְאיָּיתוֹ הַיִּיחוּדִית שֶׁל אוֹרְווּל אֶת מַעֲרֶכֶת הַיְּחָסִים שֶׁבֵּין הַיָּחִיד לַחֶבְרָה, אֶת 
רצוי:  רְאִיָּיתוֹ הַיִּיחוּדִית שֶׁל אוֹרְוֶול אֶת מַעֲרֶכֶת הַיְּחָסִים שֶׁבּ

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])